In [1]:
import pandas as pd 
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import os
from pathlib import Path

In [2]:
PROJECT_ROOT = Path.cwd().parent
os.chdir(PROJECT_ROOT)

In [4]:
X_train = pd.read_csv('data/processed/X_train.csv')
X_test = pd.read_csv('data/processed/X_test.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

In [6]:
X_train_dt = X_train.copy()
X_test_dt  = X_test.copy()

In [7]:
X_train_dt = X_train_dt.drop(columns=["Name", "Ticket"])
X_test_dt  = X_test_dt.drop(columns=["Name", "Ticket"])

In [14]:
X_train_dt['Age'] = X_train_dt['Age'].fillna(X_train_dt['Age'].median())
X_test_dt['Age'] = X_test_dt['Age'].fillna(X_train_dt['Age'].median())
X_train_dt['Embarked'] = X_train_dt['Embarked'] .fillna(
    X_train_dt["Embarked"].mode()[0]
)
X_test_dt['Embarked'] = X_test_dt['Embarked'] .fillna(
    X_train_dt["Embarked"].mode()[0]
)

In [15]:
#защита ,что бы не падл catboost при отсутсвие колонок
cat_cols = [c for c in ['Sex','Embarked'] if c in X_train_dt.columns]
#индексируем колонки
cat_features = [X_train_dt.columns.get_loc(c) for c in cat_cols]

In [19]:
model = CatBoostClassifier(
    iterations =500,
    learning_rate=0.05,
    depth =6,
    loss_function ='Logloss',
    eval_metric = 'AUC',
    random_seed = 42,
    verbose = 100
)

In [20]:
print(cat_features)

[1, 6]


In [21]:
model.fit(
    X_train_dt,y_train,
    cat_features=cat_features,
    eval_set = (X_test_dt ,y_test),
    use_best_model = True
)

0:	test: 0.8162055	best: 0.8162055 (0)	total: 162ms	remaining: 1m 20s
100:	test: 0.8509881	best: 0.8563241 (20)	total: 1.41s	remaining: 5.56s
200:	test: 0.8507246	best: 0.8563241 (20)	total: 2.78s	remaining: 4.14s
300:	test: 0.8480896	best: 0.8563241 (20)	total: 4.19s	remaining: 2.77s
400:	test: 0.8424901	best: 0.8563241 (20)	total: 5.58s	remaining: 1.38s
499:	test: 0.8360343	best: 0.8563241 (20)	total: 7s	remaining: 0us

bestTest = 0.8563241107
bestIteration = 20

Shrink model to first 21 iterations.


In [22]:
y_pred = model.predict(X_test_dt)

In [24]:
accuracy_score (y_test,y_pred)

0.7932960893854749

In [25]:
confusion_matrix(y_test, y_pred)

array([[101,   9],
       [ 28,  41]])

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.92      0.85       110
           1       0.82      0.59      0.69        69

    accuracy                           0.79       179
   macro avg       0.80      0.76      0.77       179
weighted avg       0.80      0.79      0.79       179



## Сравнение baseline-моделей

Модель CatBoost показала небольшое, но устойчивое улучшение
качества классификации по сравнению с базовой моделью на основе
дерева решений. Значение accuracy увеличилось примерно на 2%,
при этом улучшилась точность распознавания класса «выжил».
В то же время значение recall для данного класса осталось на
сопоставимом уровне, что указывает на то, что основная часть
дискриминирующего сигнала уже была извлечена простой моделью.

Данный результат свидетельствует о том, что без дополнительного
feature engineering преимущество более сложной модели ограничено.
